In [ ]:
import torch
import torch.profiler

# 模拟一个简单的模型训练过程
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = torch.nn.Linear(1024, 1024)

    def forward(self, x):
        return self.fc(x)

# 生成一个随机输入
input_tensor = torch.randn(16, 1024).to("cuda")  # 16个样本，1024个特征，使用GPU

# 初始化模型并将其移动到GPU
model = SimpleModel().to("cuda")

# 使用PyTorch Profiler来监测显存
with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA
    ],
    schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
    on_trace_ready=torch.profiler.tensorboard_trace_handler('./log'),  # 将结果保存到指定文件夹
    record_shapes=True,
    profile_memory=True,  # 开启内存分析
    with_stack=True
) as profiler:

    # 进行一些训练步骤以监测显存
    for step in range(10):
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
        optimizer.zero_grad()

        # 前向传播
        output = model(input_tensor)

        # 计算损失
        loss = output.sum()

        # 反向传播
        loss.backward()

        # 更新权重
        optimizer.step()

        # 打印当前显存使用情况
        print(f"Step {step}: {torch.cuda.memory_allocated() / 1024**2} MB allocated")

        # 记录 Profiler 数据
        profiler.step()

print("Profiling complete.")